<a href="https://colab.research.google.com/github/AndreyBuyanov/HPC.Lab3/blob/main/HPC_Lab3_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Отчёт https://disk.yandex.ru/i/5cJmnSPjYWXD5Q

In [ ]:
import os
os.environ['APACHE_SPARK_DISTR_NAME'] = 'spark-3.0.2-bin-hadoop2.7'
os.environ['APACHE_SPARK_URL'] = 'https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.0.2-bin-hadoop2.7'
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"andreew","key":"9237cca134821160d9d733ffd68a809c"}'}

In [ ]:
%%bash
apt-get install openjdk-11-jdk-headless -qq > /dev/null
[ ! -d $APACHE_SPARK_DISTR_NAME ] \
   && wget -q $APACHE_SPARK_URL \
   && tar xf $APACHE_SPARK_DISTR_NAME.tgz \
   && rm -f $APACHE_SPARK_DISTR_NAME.tgz
pip install findspark > /dev/null
mkdir /root/.kaggle
cp kaggle.json /root/.kaggle
chmod 600 /root/.kaggle/kaggle.json
[ ! -f FIFA.csv ] \
    && kaggle datasets download -d rgupta09/world-cup-2018-tweets \
    && unzip -q world-cup-2018-tweets \
    && rm -f world-cup-2018-tweets.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
debug = False
def DbgPrint(log):
    if debug:
        print(log)

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName("HPC.Lab3").getOrCreate()

In [ ]:
main_df = spark.read.csv('FIFA.csv', header=True, multiLine=True, escape="\"")
main_df.show(truncate=False)

+-------------------+----+-------------------+-------------------+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------+-----+----+-------------------------------------------------------+---------------------------------------------+----------------------------------------------------------+----------------------+----------------------+---------+-------+
|ID                 |lang|Date               |Source             |len|Orig_Tweet                                                                                                                                                            |Tweet                                                                                                                    |Likes|RTs |Hashtags                          

1. Получить пять пользователей, которых упоминали в твитах чаще всего. 

In [ ]:
def sort_dict_by_value(dict, reverse=True):
    return {key: value for key, value in sorted(dict.items(), key=lambda x: x[1], reverse=reverse)}

from itertools import islice
def take(n, iterable):
    return list(islice(iterable, n))

def print_line(length=30):
    print('-' * length)

В строке может находиться несколько пользователей. Поэтому разделяем их, превращая в список. Затем создаём RDD, в строках которого содержится только одно имя пользователя

In [ ]:
user_mention_id_rdd = main_df.rdd \
    .filter(lambda x: x['UserMentionID'] != None) \
    .flatMap(lambda x: x['UserMentionID'].split(','))

Создаём словарь, ключами которого являются имена пользователя, а значения - количество записей, соответствующих имени пользователя.

In [ ]:
users_by_mentions = user_mention_id_rdd.countByValue()

Сортируем полученный словарь по количеству записей.

In [ ]:
users_by_mentions_sorted = sort_dict_by_value(users_by_mentions)

Выбираем из словаря первые 5 записей. Таким образом получим 5 пользователей, которых упоминали чаще всего.

In [ ]:
users_count = 5
top_users_with_mentions = take(users_count, users_by_mentions_sorted.items())
top_user_names = []
for user, mentions in top_users_with_mentions:
    top_user_names.append(user)
    print(f'User: {user}, mentions: {mentions}')

User: FIFAWorldCup, mentions: 91122
User: KhaledBeydoun, mentions: 17507
User: weareoneEXO, mentions: 14974
User: HNS_CFF, mentions: 11836
User: paulpogba, mentions: 11465


2. Для каждого из этих пользователей определить десять наиболее часто употребляемых вместе с ними хештегов.

In [ ]:
excluded_hashtag = 'WorldCup'
hashtags_rdd = main_df.rdd \
    .filter(lambda x: x['UserMentionID'] != None and x['Hashtags'] != None) \
    .flatMap(lambda x: [(users, hashtags) for users in x['UserMentionID'].split(',') for hashtags in x['Hashtags'].split(',')]) \
    .filter(lambda x: x[1].lower() != excluded_hashtag.lower()) # exclude WorldCup, Worldcup and worldcup

In [ ]:
def get_user_top_hashtags(rdd, user_name, top_hashtags_count=10):
    user_hashtags = rdd \
        .filter(lambda x: x[0] == user_name) \
        .map(lambda x: x[1]) \
        .countByValue()
    user_hashtags_sorted = sort_dict_by_value(user_hashtags)
    return take(top_hashtags_count, user_hashtags_sorted.items())

In [ ]:
users_with_top_hashtags = []
DbgPrint(f'All users: {", ".join(top_user_names)}')
for user_name in top_user_names:
    DbgPrint(f'Current user: {user_name}')
    user_with_top_hashtags = get_user_top_hashtags(hashtags_rdd, user_name)
    users_with_top_hashtags.append((user_name, user_with_top_hashtags))

In [ ]:
print_line()
for user_name, top_hashtags in users_with_top_hashtags:
    print(f'User: {user_name}')
    for hashtag, count in top_hashtags:
        print(f'Hashtag: {hashtag}, count {count}')
    print_line()

------------------------------
User: FIFAWorldCup
Hashtag: FRA, count 24786
Hashtag: FIFAStadiumDJ, count 15525
Hashtag: WorldCupFinal, count 13538
Hashtag: FRACRO, count 9145
Hashtag: PowerByEXO, count 8892
Hashtag: CRO, count 8524
Hashtag: BEL, count 6442
Hashtag: FRABEL, count 5925
Hashtag: EXO, count 4774
Hashtag: CROENG, count 3631
------------------------------
User: KhaledBeydoun
Hashtag: France, count 76
Hashtag: Belgium, count 12
Hashtag: FRA, count 2
Hashtag: African, count 1
Hashtag: sports, count 1
Hashtag: arts, count 1
Hashtag: music, count 1
Hashtag: technology, count 1
Hashtag: ideas, count 1
Hashtag: Mbappe, count 1
------------------------------
User: weareoneEXO
Hashtag: FIFAStadiumDJ, count 11177
Hashtag: PowerByEXO, count 7997
Hashtag: EXO, count 5538
Hashtag: ??, count 3728
Hashtag: dafBAMA2018EXO, count 1342
Hashtag: Copa2018, count 562
Hashtag: FIFAStadiumDj, count 521
Hashtag: POWER, count 469
Hashtag: ChanyeolLeague, count 367
Hashtag: LayLeague, count 358
---

3. Выбрать из них пользователя, у которого хештеги наиболее отличается от четырёх остальных.

Поочерёдно найдём общие теги у первого пользователя и у остальных.

In [ ]:
first_user_name, first_user_hashtags_with_count = users_with_top_hashtags[0]
first_user_hashtags = [hashtag for hashtag, count in first_user_hashtags_with_count]
common_hashtags = []
for user_name, hashtags_with_count in users_with_top_hashtags[1:]:
    user_hashtags = [hashtag for hashtag, count in hashtags_with_count]
    current_user_common_hashtags = list(set(first_user_hashtags) & set(user_hashtags))
    print(f'Common hashtags for user {first_user_name} and user {user_name}: {", ".join(current_user_common_hashtags)}')
    common_hashtags.append(((first_user_name, user_name), current_user_common_hashtags))

Common hashtags for user FIFAWorldCup and user KhaledBeydoun: FRA
Common hashtags for user FIFAWorldCup and user weareoneEXO: FIFAStadiumDJ, PowerByEXO, EXO
Common hashtags for user FIFAWorldCup and user HNS_CFF: CRO, WorldCupFinal, CROENG
Common hashtags for user FIFAWorldCup and user paulpogba: BEL, CRO, WorldCupFinal, FRA


Таким образом, будем считать, что пользователь, теги которого наиболее отличаются от других, это тот, у которого меньше всего общих тегов со всеми остальными. Полученный список содержит кортежи, в которых содержится кортеж из имён двух пользователей и список их общих тегов. Отсортируем список по длинне общих тегов и в первом элементе списка будет содержаться элемент с наименьшим количеством общих тегов, что и будет являтся решением задачи.

In [ ]:
common_hashtags.sort(key=lambda x: len(x[1]))
(first_user_name, user_name), hashtags = common_hashtags[0]
print(f'The user named "{user_name}" has the most different hashtags. Hashtags: {", ".join(hashtags)}')

The user named "KhaledBeydoun" has the most different hashtags. Hashtags: FRA


4. Выбрать два пользователя, у которых наиболее близкое друг к другу соотношение частоты встречаемости выделенных хештегов.

In [ ]:
all_hashtags_with_count = hashtags_rdd \
    .map(lambda x: x[1]) \
    .countByValue()
all_hashtags_with_count_sorted = sort_dict_by_value(all_hashtags_with_count)
top10_hashtags_with_count = take(10, all_hashtags_with_count_sorted.items())
top10_hashtags = [hashtag for hashtag, count in top10_hashtags_with_count]
print(f'Top 10 hashtags: {", ".join(top10_hashtags)}')

Top 10 hashtags: FRA, CRO, FIFAStadiumDJ, WorldCupFinal, ENG, PowerByEXO, FRAARG, FRABEL, EXO, BEL


In [ ]:
excluded_user = 'FIFAWorldCup'
exclude_user = True
users_with_frequency = {}
for hashtag in top10_hashtags:
    users_count = hashtags_rdd \
        .filter(lambda x: x[1] == hashtag) \
        .filter(lambda x: x[0] != excluded_user if exclude_user else True) \
        .map(lambda x: x[0]) \
        .countByValue()
    users_count_sorted = sort_dict_by_value(users_count)
    users_with_frequency[hashtag] = take(2, users_count_sorted.items())
    DbgPrint(users_with_frequency[hashtag])  

In [ ]:
print_line()
for hashtag, data in users_with_frequency.items():
    user1, frequency1 = data[0]
    user2, frequency2 = data[1]
    print(f'Hashtag: {hashtag}')
    print(f'User 1: {user1}, frequency {frequency1} --> User 2: {user2}, frequency {frequency2}')
    print_line()

------------------------------
Hashtag: FRA
User 1: paulpogba, frequency 2890 --> User 2: ManUtd, frequency 2634
------------------------------
Hashtag: CRO
User 1: HNS_CFF, frequency 8411 --> User 2: lukamodric10, frequency 4669
------------------------------
Hashtag: FIFAStadiumDJ
User 1: weareoneEXO, frequency 11177 --> User 2: BTS_twt, frequency 2170
------------------------------
Hashtag: WorldCupFinal
User 1: FrenchTeam, frequency 3214 --> User 2: SpursOfficial, frequency 1717
------------------------------
Hashtag: ENG
User 1: HKane, frequency 6379 --> User 2: BBCMOTD, frequency 1348
------------------------------
Hashtag: PowerByEXO
User 1: weareoneEXO, frequency 7997 --> User 2: justexoandexo12, frequency 664
------------------------------
Hashtag: FRAARG
User 1: fussballerisch, frequency 1773 --> User 2: Football__Tweet, frequency 1341
------------------------------
Hashtag: FRABEL
User 1: BelRedDevils, frequency 1560 --> User 2: FrenchTeam, frequency 1307
-------------------